## 2019년 부터 2023년까지 전체 기사에 대한 LDA 모델 적용

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import os
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [3]:
# 합칠 csv파일 확인
file_list = os.listdir('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/')
file_list_csv = [file for file in file_list if file.endswith('.csv')]
print('대상 파일 리스트: {}'.format(file_list_csv))

대상 파일 리스트: ['2019년 기사(완료).csv', '2020년 기사(완료).csv', '2021년 기사(완료).csv', '2023년 기사(완료).csv', '2022년 기사(완료).csv']


In [4]:
# csv 합치기
article_data = {}
for article in file_list_csv:
  article_data[article] = pd.read_csv('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/{}'.format(article))

df = pd.concat(article_data.values(), ignore_index=True)
df.drop(columns=['Unnamed: 0','title','content','tokenize_title'],inplace=True) # LDA대상은 기사내용(content)만으로 함
df.reset_index(drop=True, inplace=True)
df

,date,url,category,tokenize_content
0,2019-01-01T07:31:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['블록', '체인', '시장', '진단', '사실', '앞', '겸손한', '정통..."
1,2019-01-01T13:15:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['종목', '상담', '카카오', '방송', '된', '국민', '주식', '고충..."
2,2019-01-02T04:03:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['재계', '뛴다', '카카오', '블록', '체인', '기술', '열사', '라..."
3,2019-01-02T06:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['카카오', '카카오', '프렌즈', '앞세운', '캐릭터', '사업', '본격'..."
4,2019-01-02T09:41:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['배우', '매니지먼트', '개사', '광고', '모델', '캐스팅', '에이전시..."
...,...,...,...,...
32174,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,IT/과학,"['간', '이동', '트렌드', '분석', '해외', '여행', '수요', '회복..."
32175,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,IT/과학,"['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지..."
32176,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,사회,"['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ..."
32177,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['카카오', '서비스', '장애', '발생', '이후', '보상', '내놓았다',..."


In [5]:
# 'tokenize_content'의 type이 str이므로 이를 list로 변환하기 위함
def str_list(token):
  str_list = token
  str_list = str_list.replace("[","").replace("]","").replace("'","").replace(" ","")
  result = str_list.split(",")
  return result

In [6]:
# content에 대한 LDA모델 적용
content_docs = []
for doc in df['tokenize_content']:
  content_docs.append(str_list(doc))
content_docs

Output hidden; open in https://colab.research.google.com to view.

In [7]:
id2word_content = corpora.Dictionary(content_docs)
print(id2word_content)

Dictionary<76391 unique tokens: ['가기', '가는', '가늠', '가능성', '가능하도록']...>


In [8]:
for value in id2word_content:
  print(value, id2word_content[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
71391 꺼려하지만
71392 박홍
71393 정몽혁
71394 진종
71395 롱코
71396 배출권거래제
71397 슈어런
71398 대쉬보드
71399 숨쉬기
71400 계략
71401 골칫
71402 황재헌
71403 강원대학교
71404 랜들
71405 임함
71406 쉬자
71407 걷히
71408 도정
71409 비엠아이
71410 오설록
71411 지에스건설
71412 확득
71413 기다려주시는
71414 케해
71415 환혼
71416 보섬
71417 같아지도록
71418 어슈어런스
71419 에몬스
71420 에몬스홈
71421 프레디
71422 싱크로나이즈
71423 앉아있어도
71424 틸팅락
71425 지웠다
71426 후려쳐야
71427 궁금해졌다는
71428 단단해졌을
71429 오두막
71430 외로웠던
71431 지내면
71432 쫓겨서
71433 채워지니까
71434 매달아서
71435 주렁주렁
71436 않았나라는
71437 외로웠었다
71438 지난날
71439 키워주며
71440 걸작
71441 장웅
71442 다졌다며
71443 김정현
71444 위에브
71445 나가는데도
71446 박광식
71447 압구정로
71448 언주로
71449 영동대로
71450 납금
71451 리세
71452 가있어하나
71453 김성아
71454 까먹는다
71455 나타나서
71456 냅다
71457 노션
71458 뛰어다니면서
71459 뛰어주실
71460 만만하지만
71461 모여있을
71462 물렁하다
71463 박만
71464 받았다던지
71465 받으셔야
71466 배움이었다
71467 배웠는지도
71468 불합격
71469 비오는
71470 비저닝
71471 송경찬
71472 쌓아가면서
71473 써놓는다거나
71474 써야하는
71475 앉아있어야
71476 어땠다는
71477 어려워하시는
71478 유기체
71479 윤예
71480 이연주
71481 저릴
7

In [9]:
corpus_TDM = []
for doc in content_docs:
  result = id2word_content.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 50 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_content, num_topics=n)

for t in lda.print_topics():
  print(t[0],":",t[1])

39 : 0.026*"소상" + 0.022*"공인" + 0.020*"근무" + 0.019*"지원" + 0.018*"판교" + 0.016*"성남시" + 0.015*"상공" + 0.010*"오피스" + 0.008*"정통부" + 0.006*"경기"
19 : 0.005*"매출" + 0.005*"페이" + 0.005*"서비스" + 0.004*"데이터" + 0.004*"협의" + 0.004*"투자" + 0.004*"게임" + 0.004*"센터" + 0.004*"주가" + 0.003*"판교"
31 : 0.015*"보수" + 0.014*"빅" + 0.014*"연봉" + 0.013*"강세" + 0.012*"회장" + 0.011*"스톡옵션" + 0.011*"부회장" + 0.010*"의장" + 0.010*"급여" + 0.009*"브레인"
43 : 0.034*"피해" + 0.017*"보상" + 0.016*"센터" + 0.015*"장애" + 0.015*"사태" + 0.011*"접수" + 0.009*"사과" + 0.008*"대책" + 0.008*"발생" + 0.007*"점검"
33 : 0.035*"분류" + 0.025*"안내" + 0.009*"원칙" + 0.008*"악재" + 0.007*"임금" + 0.006*"근로" + 0.006*"만화" + 0.006*"여력" + 0.006*"손해배상" + 0.005*"구제"
18 : 0.005*"몰아주기" + 0.005*"등급" + 0.005*"공원" + 0.003*"관광" + 0.003*"관광객" + 0.002*"자전거" + 0.002*"그린" + 0.002*"시내" + 0.002*"바이크" + 0.001*"자녀"
21 : 0.023*"아이돌" + 0.020*"라이브" + 0.019*"예능" + 0.013*"엔터테인먼트" + 0.010*"쇼핑" + 0.009*"멤버" + 0.009*"프로그램" + 0.009*"현실" + 0.007*"데뷔" + 0.007*"학습"
20 : 0.111*"화재" + 0.054*"장애" + 0.037*"복구" + 0.

In [11]:
# 토픽배정
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(content)'] = dominant_topic[0]
  df.loc[i, 'topic_details(content)'] = dominant_topic[1]
df

,date,url,category,tokenize_content,topic(content),topic_details(content)
0,2019-01-01T07:31:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['블록', '체인', '시장', '진단', '사실', '앞', '겸손한', '정통...",19.0,0.587825
1,2019-01-01T13:15:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['종목', '상담', '카카오', '방송', '된', '국민', '주식', '고충...",27.0,0.346475
2,2019-01-02T04:03:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['재계', '뛴다', '카카오', '블록', '체인', '기술', '열사', '라...",19.0,0.767109
3,2019-01-02T06:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['카카오', '카카오', '프렌즈', '앞세운', '캐릭터', '사업', '본격'...",19.0,0.419112
4,2019-01-02T09:41:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['배우', '매니지먼트', '개사', '광고', '모델', '캐스팅', '에이전시...",19.0,0.492353
...,...,...,...,...,...,...
32174,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,IT/과학,"['간', '이동', '트렌드', '분석', '해외', '여행', '수요', '회복...",19.0,0.491106
32175,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,IT/과학,"['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지...",19.0,0.424623
32176,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,사회,"['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ...",19.0,0.488608
32177,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['카카오', '서비스', '장애', '발생', '이후', '보상', '내놓았다',...",43.0,0.403719


In [12]:
# 토픽별 비중 확인
df['topic(content)'].unique()

array([19., 27.,  2., 43., 21., 47., 28., 39., 35.,  8., 33., 31., 20.,
       14., 42., 32., 29., 23., 37.,  0., 24., 48., 26.,  6.])

In [13]:
df['topic(content)'].value_counts()

19.0    19141
43.0     3153
27.0     2621
28.0     2289
2.0      1995
47.0      686
39.0      603
8.0       541
35.0      243
29.0      192
20.0      171
31.0      166
33.0      116
14.0      110
32.0       36
21.0       36
0.0        22
42.0       18
24.0       17
6.0         9
23.0        6
26.0        6
37.0        1
48.0        1
Name: topic(content), dtype: int64

In [14]:
# CSV 파일로 저장
csv_filename = "/content/drive/MyDrive/2023년 10월-11월 프로젝트/LDA모델 적용/news_전체.csv"
df.to_csv(csv_filename, index=False)

## 시각화

In [15]:
!pip install pyLDAvis==3.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 30.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

In [16]:
!pip install pandas==1.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.2
    Uninstalling pandas-2.1.2:
      Successfully uninstalled pandas-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.5.1 which is incompatible.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.1 which is incompatible.


In [17]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus_TFIDF, id2word_content)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
